In [1]:
import configparser
import pandas as pd
import datetime
import boto3
import os
import json

In [2]:
import aws_earth_fns

In [3]:
from aws_earth_fns import fetch_messages

In [4]:
def get_messages_from_queue(queue_url, my_config):
    """Generates messages from an SQS queue.
    
    Note: this continues to generate messages until the queue is empty.
    Every message on the queue will be deleted.
    
    Reference: https://alexwlchan.net/2018/01/downloading-sqs-queues/
    
    Parameters
    ----------
    queue_url : str
        URL of the SQS queue to drain.
    my_config : obj:config
        Configuration containing aws access parameters

    """
    # set values from config
    aws_access_key_id = my_config['aws']['aws_access_key_id']
    aws_secret_access_key = my_config['aws']['aws_secret_access_key']
    region_name = my_config['aws']['region_name']
    
    client = boto3.client(
        'sqs',
        aws_access_key_id = aws_access_key_id,
        aws_secret_access_key = aws_secret_access_key,
        region_name = region_name)
    while True:
        resp = client.receive_message(
            QueueUrl=queue_url,
            AttributeNames=['All'],
            MaxNumberOfMessages=10
        )

        try:
            yield from resp['Messages']
        except KeyError:
            return

        entries = [
            {'Id': msg['MessageId'], 'ReceiptHandle': msg['ReceiptHandle']}
            for msg in resp['Messages']
        ]

        resp = client.delete_message_batch(
            QueueUrl=queue_url, Entries=entries
        )

        if len(resp['Successful']) != len(entries):
            raise RuntimeError(
                f"Failed to delete messages: entries={entries!r} resp={resp!r}"
            )

In [5]:
def main_download():
    # read config
    my_config = configparser.ConfigParser()
    my_config.read('C:/Users/Dan Travers/Documents/GitHub/aws_earth_download/src/aws_config.ini')
    # set values from config
    variables = [x.strip() for x in my_config['aws']['variables'].split(',')]
    ukv_store_path = my_config['local_config']['ukv_store_path']
    mogreps_uk_store_path = my_config['local_config']['mogreps_uk_store_path']
    
    # download MOGREPS messages
    mdf = pd.DataFrame([])
    for mes in get_messages_from_queue(my_config['aws']['mogreps_url'], my_config):    
        obj = json.loads(json.loads(mes['Body'])['Message'])
        if (obj['name'] in variables) & \
        (datetime.datetime.strptime(obj['forecast_reference_time'], "%Y-%m-%dT%H:%M:%SZ").hour in range(3, 15)) & \
        (datetime.datetime.strptime(obj['time'], "%Y-%m-%dT%H:%M:%SZ").hour in range(7, 19)):
            print(obj['name'], obj['forecast_reference_time'], obj['time'], int(obj['forecast_period'])/3600/24)
            mdf = mdf.append(pd.DataFrame(obj, index=[mes['MessageId']]))
    save_location = os.path.join(mogreps_uk_store_path, \
        'mogreps_sqs_messages_{}.csv'.format(datetime.datetime.now().strftime("%Y-%m-%dT%H-%M")))
    mdf.to_csv(save_location)
    
    # download UK-V messages
    mdf = pd.DataFrame([])
    for mes in get_messages_from_queue(my_config['aws']['ukv_url'], my_config):    
        obj = json.loads(json.loads(mes['Body'])['Message'])
        if (obj['name'] in variables) & \
        (datetime.datetime.strptime(obj['forecast_reference_time'], "%Y-%m-%dT%H:%M:%SZ").hour in range(3, 7)) & \
        (datetime.datetime.strptime(obj['time'], "%Y-%m-%dT%H:%M:%SZ").hour in range(7, 19)):
            print(obj['name'], obj['forecast_reference_time'], obj['time'], int(obj['forecast_period'])/3600/24)
            mdf = mdf.append(pd.DataFrame(obj, index=[mes['MessageId']]))
    save_location = os.path.join(ukv_store_path, \
        'ukv_sqs_messages_{}.csv'.format(datetime.datetime.now().strftime("%Y-%m-%dT%H-%M")))
    mdf.to_csv(save_location)

In [6]:
main_download()

air_temperature 2019-05-15T04:00:00Z 2019-05-18T13:00:00Z 3.375
air_temperature 2019-05-15T04:00:00Z 2019-05-18T13:00:00Z 3.375
air_temperature 2019-05-15T04:00:00Z 2019-05-18T09:00:00Z 3.2083333333333335


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


air_temperature 2019-05-15T04:00:00Z 2019-05-18T17:00:00Z 3.5416666666666665
air_temperature 2019-05-15T04:00:00Z 2019-05-18T17:00:00Z 3.5416666666666665
air_temperature 2019-05-15T04:00:00Z 2019-05-18T15:00:00Z 3.4583333333333335
air_temperature 2019-05-15T04:00:00Z 2019-05-18T12:00:00Z 3.3333333333333335
air_temperature 2019-05-15T04:00:00Z 2019-05-18T11:00:00Z 3.2916666666666665
air_temperature 2019-05-15T04:00:00Z 2019-05-18T08:00:00Z 3.1666666666666665
air_temperature 2019-05-15T04:00:00Z 2019-05-18T10:00:00Z 3.25
air_temperature 2019-05-15T04:00:00Z 2019-05-18T13:00:00Z 3.375
air_temperature 2019-05-15T04:00:00Z 2019-05-18T12:00:00Z 3.3333333333333335
air_temperature 2019-05-15T04:00:00Z 2019-05-18T08:00:00Z 3.1666666666666665
air_temperature 2019-05-15T04:00:00Z 2019-05-18T16:00:00Z 3.5
wind_from_direction 2019-05-15T04:00:00Z 2019-05-18T14:00:00Z 3.4166666666666665
air_temperature 2019-05-15T04:00:00Z 2019-05-18T16:00:00Z 3.5
air_temperature 2019-05-15T04:00:00Z 2019-05-18T17:0

air_temperature 2019-05-15T04:00:00Z 2019-05-18T10:00:00Z 3.25
air_temperature 2019-05-15T04:00:00Z 2019-05-18T14:00:00Z 3.4166666666666665
wind_speed 2019-05-15T04:00:00Z 2019-05-18T08:00:00Z 3.1666666666666665
wind_from_direction 2019-05-15T04:00:00Z 2019-05-18T17:00:00Z 3.5416666666666665
wind_from_direction 2019-05-15T04:00:00Z 2019-05-18T18:00:00Z 3.5833333333333335
wind_from_direction 2019-05-15T04:00:00Z 2019-05-18T09:00:00Z 3.2083333333333335
wind_from_direction 2019-05-15T04:00:00Z 2019-05-18T16:00:00Z 3.5
wind_from_direction 2019-05-15T04:00:00Z 2019-05-18T08:00:00Z 3.1666666666666665
wind_speed 2019-05-15T04:00:00Z 2019-05-18T17:00:00Z 3.5416666666666665
air_temperature 2019-05-15T04:00:00Z 2019-05-18T15:00:00Z 3.4583333333333335
wind_from_direction 2019-05-15T04:00:00Z 2019-05-18T12:00:00Z 3.3333333333333335
wind_speed 2019-05-15T04:00:00Z 2019-05-18T10:00:00Z 3.25
relative_humidity 2019-05-15T04:00:00Z 2019-05-18T13:00:00Z 3.375
air_temperature 2019-05-15T04:00:00Z 2019-05

wind_speed 2019-05-15T04:00:00Z 2019-05-19T10:00:00Z 4.25
surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T04:00:00Z 2019-05-19T16:00:00Z 4.5
wind_speed 2019-05-15T04:00:00Z 2019-05-19T15:00:00Z 4.458333333333333
wind_speed 2019-05-15T04:00:00Z 2019-05-19T09:00:00Z 4.208333333333333
wind_speed 2019-05-15T04:00:00Z 2019-05-19T11:00:00Z 4.291666666666667
wind_speed 2019-05-15T04:00:00Z 2019-05-19T17:00:00Z 4.541666666666667
surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T04:00:00Z 2019-05-19T15:00:00Z 4.458333333333333
wind_speed 2019-05-15T04:00:00Z 2019-05-19T09:00:00Z 4.208333333333333
wind_speed 2019-05-15T04:00:00Z 2019-05-19T15:00:00Z 4.458333333333333
air_temperature 2019-05-15T04:00:00Z 2019-05-19T07:00:00Z 4.125
surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T04:00:00Z 2019-05-19T08:00:00Z 4.166666666666667
wind_speed 2019-05-15T04:00:00Z 2019-05-19T17:00:00Z 4.541666666666667
wind_from_direction 2019-05-15T04:00:00Z 2019-05-19T11:00:00Z 4.2916

...

surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T14:00:00Z 2019-05-19T18:00:00Z 4.166666666666667
surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T13:00:00Z 2019-05-20T07:00:00Z 4.75
wind_from_direction 2019-05-15T13:00:00Z 2019-05-20T11:00:00Z 4.916666666666667
wind_speed 2019-05-15T14:00:00Z 2019-05-19T14:00:00Z 4.0
wind_from_direction 2019-05-15T13:00:00Z 2019-05-20T15:00:00Z 5.083333333333333
wind_from_direction 2019-05-15T14:00:00Z 2019-05-19T18:00:00Z 4.166666666666667
wind_speed 2019-05-15T14:00:00Z 2019-05-19T17:00:00Z 4.125
relative_humidity 2019-05-15T14:00:00Z 2019-05-19T11:00:00Z 3.875
air_temperature 2019-05-15T14:00:00Z 2019-05-19T14:00:00Z 4.0
air_temperature 2019-05-15T13:00:00Z 2019-05-20T17:00:00Z 5.166666666666667
wind_from_direction 2019-05-15T13:00:00Z 2019-05-20T17:00:00Z 5.166666666666667
surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T13:00:00Z 2019-05-20T08:00:00Z 4.791666666666667
air_temperature 2019-05-15T14:00:00Z 2019-05-2

air_temperature 2019-05-15T14:00:00Z 2019-05-20T13:00:00Z 4.958333333333333
air_temperature 2019-05-15T14:00:00Z 2019-05-18T08:00:00Z 2.75
relative_humidity 2019-05-15T14:00:00Z 2019-05-18T07:00:00Z 2.7083333333333335
wind_speed 2019-05-15T14:00:00Z 2019-05-20T17:00:00Z 5.125
relative_humidity 2019-05-15T14:00:00Z 2019-05-20T07:00:00Z 4.708333333333333
relative_humidity 2019-05-15T14:00:00Z 2019-05-20T18:00:00Z 5.166666666666667
wind_from_direction 2019-05-15T14:00:00Z 2019-05-20T13:00:00Z 4.958333333333333
relative_humidity 2019-05-15T14:00:00Z 2019-05-20T11:00:00Z 4.875
air_temperature 2019-05-15T14:00:00Z 2019-05-18T12:00:00Z 2.9166666666666665
surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T14:00:00Z 2019-05-18T10:00:00Z 2.8333333333333335
relative_humidity 2019-05-15T13:00:00Z 2019-05-20T15:00:00Z 5.083333333333333
surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T13:00:00Z 2019-05-20T09:00:00Z 4.833333333333333
relative_humidity 2019-05-15T14:00:00Z 2019-05-

air_temperature 2019-05-15T14:00:00Z 2019-05-16T18:00:00Z 1.1666666666666667
wind_speed 2019-05-15T14:00:00Z 2019-05-20T16:00:00Z 5.083333333333333
wind_speed 2019-05-15T14:00:00Z 2019-05-20T15:00:00Z 5.041666666666667
wind_speed 2019-05-15T14:00:00Z 2019-05-20T12:00:00Z 4.916666666666667
wind_speed 2019-05-15T14:00:00Z 2019-05-20T13:00:00Z 4.958333333333333
wind_from_direction 2019-05-15T14:00:00Z 2019-05-20T10:00:00Z 4.833333333333333
air_temperature 2019-05-15T14:00:00Z 2019-05-20T07:00:00Z 4.708333333333333
air_temperature 2019-05-15T14:00:00Z 2019-05-17T16:00:00Z 2.0833333333333335
relative_humidity 2019-05-15T14:00:00Z 2019-05-20T17:00:00Z 5.125
wind_speed 2019-05-15T14:00:00Z 2019-05-20T11:00:00Z 4.875
wind_speed 2019-05-15T14:00:00Z 2019-05-20T08:00:00Z 4.75
relative_humidity 2019-05-15T14:00:00Z 2019-05-20T13:00:00Z 4.958333333333333
wind_from_direction 2019-05-15T14:00:00Z 2019-05-20T11:00:00Z 4.875
air_temperature 2019-05-15T14:00:00Z 2019-05-17T14:00:00Z 2.0
air_temperature

wind_speed 2019-05-15T14:00:00Z 2019-05-18T10:00:00Z 2.8333333333333335
wind_speed 2019-05-15T14:00:00Z 2019-05-18T13:00:00Z 2.9583333333333335
wind_speed 2019-05-15T14:00:00Z 2019-05-19T09:00:00Z 3.7916666666666665
relative_humidity 2019-05-15T14:00:00Z 2019-05-19T10:00:00Z 3.8333333333333335
wind_from_direction 2019-05-15T14:00:00Z 2019-05-20T17:00:00Z 5.125
wind_speed 2019-05-15T14:00:00Z 2019-05-18T09:00:00Z 2.7916666666666665
relative_humidity 2019-05-15T14:00:00Z 2019-05-18T11:00:00Z 2.875
wind_speed 2019-05-15T14:00:00Z 2019-05-18T11:00:00Z 2.875
air_temperature 2019-05-15T14:00:00Z 2019-05-18T16:00:00Z 3.0833333333333335
surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T14:00:00Z 2019-05-18T13:00:00Z 2.9583333333333335
wind_from_direction 2019-05-15T14:00:00Z 2019-05-18T09:00:00Z 2.7916666666666665
wind_speed 2019-05-15T14:00:00Z 2019-05-18T10:00:00Z 2.8333333333333335
wind_from_direction 2019-05-15T14:00:00Z 2019-05-18T13:00:00Z 2.9583333333333335
air_temperature 201

wind_speed 2019-05-15T14:00:00Z 2019-05-20T08:00:00Z 4.75
wind_from_direction 2019-05-15T14:00:00Z 2019-05-20T09:00:00Z 4.791666666666667
air_temperature 2019-05-15T14:00:00Z 2019-05-20T13:00:00Z 4.958333333333333
air_temperature 2019-05-15T14:00:00Z 2019-05-20T14:00:00Z 5.0
surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T14:00:00Z 2019-05-20T15:00:00Z 5.041666666666667
surface_direct_downwelling_shortwave_flux_in_air 2019-05-15T14:00:00Z 2019-05-20T12:00:00Z 4.916666666666667
wind_from_direction 2019-05-15T14:00:00Z 2019-05-20T15:00:00Z 5.041666666666667
wind_speed 2019-05-15T14:00:00Z 2019-05-20T13:00:00Z 4.958333333333333
relative_humidity 2019-05-15T14:00:00Z 2019-05-20T11:00:00Z 4.875
